In [1]:
import xarray as xr
import numpy as np
import netCDF4 as nc4
import pandas as pd
import os
import glob
import uuid

(SSBN7 / SUN2WAVE / SUN2W) Sunset Nearshore Wave
https://stage.admin.axds.co/#!/sensors/metadata/stations/view?stationId=100058&tab=data

```
ADCP: Current speed and dir at 0,-10m (1000360, 1000356)

device_1000360.nc
	time = 1011 ;
	z = 2 ;
device_1000356.nc
	time = 1011 ;
	z = 2 ;

ADCP: Water temp at -10m (1000361)

device_1000361.nc
	time = 1011 ;
	z = 1 ;
    
Waves at surface: wave height, wave period, wind direction (1000357, 1000359, 1000358)

device_1000357.nc
	time = 1948 ;
	z = 1 ;
device_1000358.nc
	time = 338 ;
	z = 1 ;
device_1000359.nc
	time = 1948 ;
	z = 1 ;
```


In [2]:
station_id='100058'

In [3]:
versions = [xr.__version__, np.__version__, nc4.__version__, pd.__version__ ]
versions

['0.15.1', '1.18.1', '1.5.3', '1.0.3']

In [4]:
# inspect all device files
# each one currently has time, z dimensions
device_files = sorted(glob.glob(station_id + '/device*.nc'))
print(device_files)
for f in device_files:
    print('\n'+f)
    d = nc4.Dataset(f)
    print(d)

['100058/device_1000356_current_dir.nc', '100058/device_1000357_wave_period.nc', '100058/device_1000358_wind_dir.nc', '100058/device_1000359_wave_height.nc', '100058/device_1000360_current_speed.nc', '100058/device_1000361_water_temp.nc']

100058/device_1000356_current_dir.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: feed_1000312_raw
    dimensions(sizes): time(1011), z(2)
    variables(dimensions): uint8 qc_agg_1000356(time,z), uint64 qc_tests_1000356(time,z), int32 time(time), float64 value_1000356(time,z), float64 z(z)
    groups: 

100058/device_1000357_wave_period.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: feed_1000313_raw
    dimensions(sizes): time(1948), z(1)
    variables(dimensions): uint8 qc_agg_1000357(time,z), uint64 qc_tests_1000357(time,z), int32 time(time), float64 value_1000357(time,z), float64 z(z)
    groups: 

100058/device_1000358_wind_dir.nc
<class 'netC

## Create a timeSeriesProfile with time, station dimensions

Similar to Option 2 below with time, station dimensions:

In [5]:
%%time
# combine
timeseries = xr.open_mfdataset(device_files, combine='by_coords', parallel=True)
timeseries = timeseries.rename_dims({"z": "station"})
timeseries = timeseries.reset_coords()
timeseries['station']=(['station'], [1,2])
timeseries['latitude']=(['station'], [33.8444]*2)
timeseries['longitude']=(['station'], [-78.4839]*2)

# add attributes:
timeseries['station'].attrs['cf_role'] = 'timeseries_id'
timeseries.attrs['cdm_data_type'] = 'TimeSeries'
timeseries.attrs['cdm_timeseries_variables'] = 'station,longitude,latitude,z'
timeseries.attrs['title'] = 'DSG TimeSeries'
timeseries


CPU times: user 125 ms, sys: 34.4 ms, total: 160 ms
Wall time: 145 ms


<xarray.Dataset>
Dimensions:           (station: 2, time: 2621)
Coordinates:
  * time              (time) datetime64[ns] 2018-06-30T08:00:00 ... 2020-02-18T15:52:00
  * station           (station) int64 1 2
Data variables:
    z                 (station) float64 -10.0 0.0
    qc_agg_1000356    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000356  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000356     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_agg_1000357    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000357  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000357     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_agg_1000358    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000358  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000358     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_agg_1000359    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000359  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000359     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_agg_1000360    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000360  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000360     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_agg_1000361    (time, station) float32 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    qc_tests_1000361  (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    value_1000361     (time, station) float64 dask.array<chunksize=(2621, 2), meta=np.ndarray>
    latitude          (station) float64 33.84 33.84
    longitude         (station) float64 -78.48 -78.48
Attributes:
    title:                     DSG TimeSeries
    cdm_data_type:             TimeSeries
    cdm_timeseries_variables:  station,longitude,latitude,z

Add some minimal attribution:

In [6]:
timeseries['value_1000356'].attrs['standard_name'] = 'sea_water_velocity_to_direction'
timeseries['value_1000357'].attrs['standard_name'] = 'sea_surface_wave_significant_period'
timeseries['value_1000358'].attrs['standard_name'] = 'wind_from_direction'
timeseries['value_1000359'].attrs['standard_name'] = 'sea_surface_wave_significant_height'
timeseries['value_1000360'].attrs['standard_name'] = 'sea_water_speed'
timeseries['value_1000361'].attrs['standard_name'] = 'sea_water_temperature'
timeseries['value_1000361']


<xarray.DataArray 'value_1000361' (time: 2621, station: 2)>
dask.array<where, shape=(2621, 2), dtype=float64, chunksize=(2621, 2), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2018-06-30T08:00:00 ... 2020-02-18T15:52:00
  * station  (station) int64 1 2
Attributes:
    standard_name:  sea_water_temperature

### test some values - waves (z=0 only):

In [7]:
# z=-10: this is an empty slice:
timeseries.value_1000359.loc['2018-06-30T08:00:00':'2018-06-30T12:00:00',1].compute()

<xarray.DataArray 'value_1000359' (time: 13)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
  * time     (time) datetime64[ns] 2018-06-30T08:00:00 ... 2018-06-30T12:00:00
    station  int64 1
Attributes:
    standard_name:  sea_surface_wave_significant_height

In [8]:
# z=0: this has data:
timeseries.value_1000359.loc['2018-06-30T08:00:00':'2018-06-30T12:00:00',2].compute()

<xarray.DataArray 'value_1000359' (time: 13)>
array([0.39,  nan,  nan, 0.4 ,  nan,  nan, 0.32,  nan,  nan, 0.32,  nan,
        nan, 0.35])
Coordinates:
  * time     (time) datetime64[ns] 2018-06-30T08:00:00 ... 2018-06-30T12:00:00
    station  int64 2
Attributes:
    standard_name:  sea_surface_wave_significant_height

### test some values - current speed (both z values):¶

In [9]:
# z=-10:
timeseries.value_1000360.loc['2018-06-30T08:00:00':'2018-06-30T12:00:00',1].compute()

<xarray.DataArray 'value_1000360' (time: 13)>
array([0.063, 0.073, 0.008, 0.033, 0.077, 0.083, 0.048, 0.042, 0.033,
       0.019, 0.044, 0.036, 0.07 ])
Coordinates:
  * time     (time) datetime64[ns] 2018-06-30T08:00:00 ... 2018-06-30T12:00:00
    station  int64 1
Attributes:
    standard_name:  sea_water_speed

In [10]:
# z=0:
timeseries.value_1000360.loc['2018-06-30T08:00:00':'2018-06-30T12:00:00',2].compute()

<xarray.DataArray 'value_1000360' (time: 13)>
array([0.042, 0.126, 0.079, 0.127, 0.108, 0.141, 0.088, 0.116, 0.074,
       0.064, 0.095, 0.073, 0.122])
Coordinates:
  * time     (time) datetime64[ns] 2018-06-30T08:00:00 ... 2018-06-30T12:00:00
    station  int64 2
Attributes:
    standard_name:  sea_water_speed

### To Do:
* Because each variable isn't measured at each 'station' anyway (wind, waves, water temp, etc), there are still gaps in the resulting output.  Maybe there's no way around this..



### Write out netcdf:
This will at least work for ERDDAP testing for timeSeries

In [11]:
%%time
# write to single netcdf
timeseries_filename = f"{station_id}/station_{station_id}_timeseries_{uuid.uuid4().hex}.nc"
print(timeseries_filename)
timeseries.to_netcdf(timeseries_filename)

100058/station_100058_timeseries_db645d202c2a4e3aafadadae288f8d43.nc
CPU times: user 132 ms, sys: 4.35 ms, total: 137 ms
Wall time: 130 ms


# Option 1: `timeSeriesProfile`

1) Combine datasets using timeSeriesProfile DSG feature type, with a single ‘feature’ (aka ‘station’ dimension of length=1), where:

* lat, lon, station variables vary by the ‘station’ dimension.
* Z dimension would have a size equal to the number of different depths (2 in this case), and 
* all variables would be dimensioned by the same coordinates. This would lead to lots of missing data.
* Similar to: H.17 orthogonal multidimensional array timeSeriesProfile representation (http://cfconventions.org/cf-conventions/cf-conventions.html#_multidimensional_array_representations_of_time_series_profiles)

Example: https://coastwatch.pfeg.noaa.gov/erddap/tabledap/nmspWcosAdcpS.html (ADCP)


In [ ]:
%%time
# combine
timeseries_profile = xr.open_mfdataset(device_files, combine='by_coords', parallel=True)
timeseries_profile = timeseries_profile.expand_dims({'station': ['sun2wave']})
timeseries_profile['latitude'] = 33.8444
timeseries_profile['longitude'] = -78.4839
timeseries_profile['station'].attrs['cf_role'] = 'timeseries_id'
timeseries_profile['time'].attrs['cf_role'] = 'profile_id'
timeseries_profile.attrs['cdm_data_type'] = 'TimeSeriesProfile'
timeseries_profile.attrs['cdm_timeseries_variables'] = 'station,longitude,latitude'
timeseries_profile.attrs['cdm_profile_variables'] = 'time'
timeseries_profile.attrs['cdm_altitude_proxy'] = 'z'    
timeseries_profile.attrs['title'] = 'DSG TimeSeriesProfile'
timeseries_profile

In [ ]:
# create a new copy:
timeseries_profile2 = timeseries_profile

In [ ]:
timeseries_profile['station']

In [ ]:
timeseries_profile['time']

In [ ]:
timeseries_profile['z']

In [ ]:
%%time
# write to single netcdf
timeseries_profile_filename = f"{station_id}/station_{station_id}_timeseries_profile_{uuid.uuid4().hex}.nc"
print(timeseries_profile_filename)
timeseries_profile.to_netcdf(timeseries_profile_filename)

# Option 2: `timeSeries` with multi-dim `station`

2) Combine datasets using a timeSeries DSG format with multiple timeSeries ‘features’ per-sensor location (aka ‘station’ dimension of length=2 in this case), where:

* lat, lon, Z and station variables vary by the ‘station’ dimension.  
* data variables vary by ‘station, time’ and time variable by ‘time’ dimension (assuming same sampling frequency - aka orthogonal timeSeries representation).  This should eliminate some missing data gaps in resulting ERDDAP outputs. ‘time’ variable is a coordinate variable in this case.



In [ ]:
# assign 'station' var per device
# NOTE: it's unclear how we would do this in an automated way. 
#       would we have to manually group devices that sample at similar depths?

current_dir = xr.open_dataset('100058/device_1000356_current_dir.nc')
current_dir = current_dir.expand_dims({'station': ['adcp']})

current_speed = xr.open_dataset('100058/device_1000360_current_speed.nc')
current_speed = current_speed.expand_dims({'station': ['adcp']})

water_temp = xr.open_dataset('100058/device_1000361_water_temp.nc')
water_temp = water_temp.expand_dims({'station': ['adcp_temp']})

wave_period = xr.open_dataset('100058/device_1000357_wave_period.nc')
wave_period = wave_period.expand_dims({'station': ['waves']})

wave_height = xr.open_dataset('100058/device_1000359_wave_height.nc')
wave_height = wave_height.expand_dims({'station': ['waves']})

wind_dir = xr.open_dataset('100058/device_1000358_wind_dir.nc')
wind_dir = wind_dir.expand_dims({'station': ['waves']})

timeseries = xr.combine_by_coords([current_dir, current_speed, water_temp, wave_period, wave_height, wind_dir])
timeseries['latitude'] = 33.8444
timeseries['longitude'] = -78.4839
timeseries['station'].attrs['cf_role'] = 'timeseries_id'
timeseries.attrs['cdm_data_type'] = 'TimeSeries'
timeseries.attrs['cdm_timeseries_variables'] = 'station,longitude,latitude'
timeseries.attrs['title'] = 'DSG TimeSeries'
timeseries

In [ ]:
timeseries['station']

In [ ]:
%%time
# write to single netcdf
timeseries_filename = f"{station_id}/station_{station_id}_timeseries_{uuid.uuid4().hex}.nc"
print(timeseries_filename)
timeseries.to_netcdf(timeseries_filename)